# Final Project: Walmart Sales

## Aditya Nakai, John Sullivan, Peter Zhou, Anirudh Mittal

In this Kaggle competition, we will predict the sales of 111 potentially weather-sensitive products (such as umbrellas, bread, and milk) around the time of major weather events at 45 Walmart locations. 

After doing our initial EDA, we discovered that weather has minimal effect on sales. Instead we chose to focus on other factors such as the month of the year, holidays, and weekends. However, since the focus of this Kaggle competition was on weather, we did include a storm variable that as an indicator feature of any day that received more than one inch of rain or two inches of snow. We also found out that the majority of items in the training dataset had zero units sold. Out of the possible 4995 possible store and item_nbr combinations, only 255 contained any units sold. We attributed this to the item possibly not being stocked at the stores. 

Therefore our final methodology is first removing all items that have zero units sold using the store and item_nbr combination from the initial 4,671,600 training examples. Then we trained the dataset using the month of the year, holidays, weekends, and storm as our features. We ran a linear regression to predict this against 526,917 test data points. Then we went into the prediction to change the prediction to zero if the store and item_nbr combination was zero from the training set. This gave us a Root Mean Squared Logarithmic Error of 0.4185.

Kaggle link: https://www.kaggle.com/c/walmart-recruiting-sales-in-stormy-weather

Github link: https://github.com/jdsul/w207_summer_project

In [1]:
###### test submission
holidays=['2012-01-31','2012-01-16','2012-02-20','2012-05-28','2012-07-04','2012-09-03','2012-10-08','2012-11-12','2012-11-22','2012-12-25',
'2013-01-01','2013-01-21','2013-02-18','2013-05-27','2013-07-04','2013-09-02','2013-10-14','2013-11-11','2013-11-28','2013-12-25','2014-01-01',
'2014-01-20','2014-02-17','2014-05-26','2014-07-04','2014-09-01','2014-10-10','2014-11-11','2014-11-27','2014-12-25',
]

import csv
keys_csv = open('key.csv', 'rt')
keys=csv.reader(keys_csv)
store_station={}
for store,station in keys:
    store_station[store]=station

weather_csv = open('weather.csv', 'rt')
weather=csv.reader(weather_csv)
date_station_snowfall_rain={}
for w in weather:
    stationnumber=w[0]
    date=w[1]
    snowfall=w[13]
    rain=w[14]
    date_station_snowfall_rain[date+"_"+stationnumber]=snowfall+"_"+rain

train_csv = open('train.csv', 'rt')
train=csv.reader(train_csv)
train_date_store_item_units={}
count=0
for date,store_nbr,item_nbr,units in train:
       train_date_store_item_units[date+"_"+store_nbr+"_"+item_nbr]=units
        
train_csv1 = open('train.csv', 'rt')
train1=csv.reader(train_csv1)
train=[]
for t in train1:
    train.append(t)
    
test_csv = open('test.csv', 'rt')
test=csv.reader(test_csv)
test_date_store_item_units={}
count=0
for date,store_nbr,item_nbr in test:
       test_date_store_item_units[date+"_"+store_nbr+"_"+item_nbr]=units

test_csv1 = open('test.csv', 'rt')
test1=csv.reader(test_csv1)
test=[]
for t in test1:
    test.append(t)

#Testing the score after adding in variables: Fridays, weekends, public holidays from 2012-2014

import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model

train_no_header = train[1:len(train)]


##Making a list of weather events and using the key to find assosciated store number for the weather station
store_weather_events=[] #[store_number, station number, date, snowfall,rain]
for w in weather:
#     stationnumber=w[0]
#     date=w[1]
#     snowfall=w[13]
#     rain=w[14]
    store_number='err'
    for i in range(1,46):
        if store_station[str(i)]==str(w[0]):
            store_number=i
            if w[13]=='M' or w[13]=='snowfall' or w[13]=='T' or w[13]=='  T':
                snowfall=float(0.0)
            else:
                snowfall=float(w[13])
            if w[14]=='M' or w[14]=='preciptotal' or w[14]=='T' or w[14]=='  T':
                rain=float(0.0)
            else:
                rain=float(w[14])
            store_weather_events.append([str(store_number),w[0],w[1],snowfall,rain])
store_weather_events=store_weather_events[1:]


final_storm_list=[]
for item in store_weather_events:
    if item[3]>=1 or item[3]>=2:#a weather event as any day in which more than an inch of rain or two inches of snow was observed.
        final_storm_list.append([item[0],item[2],1])
    else:
        continue
##

# add in day of the week variable where 0=Monday and 6=Sunday and every other day in between
train_new_variables = []
for x in train_no_header:
    year, month, day = (int(z) for z in x[0].split('-'))    
    day_of_week = datetime.date(year, month, day).weekday()
    day_of_year = datetime.date(year, month, day).timetuple().tm_yday
    month_of_year = datetime.date(year, month, day).month
    if day_of_week==4:
        is_friday=1
    else:
        is_friday=0
    if day_of_week in [4,5,6]:
        is_weekend=1
    else:
        is_weekend=0
    if x[0] in holidays:
        holiday_var=1
    else:
        holiday_var=0
    storm_var=0
    for item in final_storm_list:
        if item[0]==x[1] and item[1]==x[0]:
            storm_var=1
            break
    train_new_variables.append(x + [day_of_week] + [day_of_year] + [month_of_year] +[holiday_var] + [is_friday] + [is_weekend] + [storm_var])
    
# our headers for reference
header = ['date', 'store_nbr', 'item_nbr', 'units', 'day_of_week', 'day_of_year', 'month_of_year',
         'holiday', 'is_friday', 'is_weekend', 'storm']

train_columns = [1, 2, 6, 7, 9, 10]

train_data = [[each_list[i] for i in train_columns] for each_list in train_new_variables]
train_labels = [train_new_variables[i][3] for i in range(0, len(train_new_variables))]

# test set
test_no_header = test[1:len(test)]

# add in day of the week variable where 0=Monday and 6=Sunday and every other day in between
test_new_variables = []
for x in test_no_header:
    year, month, day = (int(z) for z in x[0].split('-'))    
    day_of_week = datetime.date(year, month, day).weekday()
    day_of_year = datetime.date(year, month, day).timetuple().tm_yday
    month_of_year = datetime.date(year, month, day).month
    if day_of_week==4:
        is_friday=1
    else:
        is_friday=0
    if day_of_week in [4,5,6]:
        is_weekend=1
    else:
        is_weekend=0
    if x[0] in holidays:
        holiday_var=1
    else:
        holiday_var=0
    storm_var=0
    for item in final_storm_list:
        if item[0]==x[1] and item[1]==x[0]:
            storm_var=1
            break
    test_new_variables.append(x + [day_of_week] + [day_of_year] + [month_of_year] +[holiday_var] + [is_friday] + [is_weekend] +[storm_var])
    
# our headers for reference
test_header = ['date', 'store_nbr', 'item_nbr', 'day_of_week', 'day_of_year', 'month_of_year',
         'holiday', 'is_friday', 'is_weekend', 'storm']

test_columns = [1, 2, 5, 6, 8, 9]

test_data = [[each_list[i] for i in test_columns] for each_list in test_new_variables]

# change data into floats
train_data = np.array(train_data).astype(np.float)
train_labels = np.array(train_labels).astype(np.float)
test_data = np.array(test_data).astype(np.float)

# Linear Regression using store_nbr, item_nbr, month, holiday, and weekend
lr = LinearRegression()
linear_model = lr.fit(train_data, train_labels)
prediction = linear_model.predict(test_data)

#### prediction excluding zero units sold from data

# change labels from str to int
train_labels_int = [int(train_labels[i]) for i in range(0, len(train_labels))]

# turn data to pandas df
import pandas as pd
df = pd.DataFrame(
    {header[1] : [train_data[i][0] for i in range(0, len(train_data))],
    header[2]: [train_data[i][1] for i in range(0, len(train_data))],
    header[3]: train_labels_int},
    columns = [header[1], header[2], header[3]])

# sum of units sold grouped by store and item
units_sold_by_store_and_item = df.groupby([header[1], header[2]], as_index=False).sum()

sold_list = []
for i in range(0, len(units_sold_by_store_and_item)):
    if units_sold_by_store_and_item['units'][i] != 0:
        sold_list += [[units_sold_by_store_and_item['store_nbr'][i], 
                                  units_sold_by_store_and_item['item_nbr'][i]]]

# new training dataset called data_without_zeros containing no stores and items that sold zero units
data_without_zeros=[]
for i in range(0, len(train_new_variables)):
    if [float(train_new_variables[i][1]), float(train_new_variables[i][2])] in sold_list:
        data_without_zeros += [train_new_variables[i]]

train_data_no_zeros = [[each_list[i] for i in train_columns] for each_list in data_without_zeros]
train_labels_no_zeros = [data_without_zeros[i][3] for i in range(0, len(data_without_zeros))]

# change data into floats
train_data_no_zeros = np.array(train_data_no_zeros).astype(np.float)
train_labels_no_zeros = np.array(train_labels_no_zeros).astype(np.float)

# Linear Regression using store_nbr, item_nbr, month of year, holiday, and weekend
lr_no_zeros = LinearRegression()
linear_model_no_zeros = lr_no_zeros.fit(train_data_no_zeros, train_labels_no_zeros)
prediction_no_zeros = linear_model_no_zeros.predict(test_data)

# if prediction is less than 0 change it to 0
for i in range(0, len(prediction_no_zeros)):
    if prediction_no_zeros[i] < 0:
        prediction_no_zeros[i] = 0

# if prediction is in the sold list then keep it, if not then change it to 0
for i in range(0, len(test_data)):
    if [float(test_data[i][0]), float(test_data[i][1])] in sold_list:
        continue
    else:
        prediction_no_zeros[i] = 0
        
# create submission file    
submission = []
for i in range(0, len(prediction_no_zeros)):
    submission += [[test_new_variables[i][1] + "_" + test_new_variables[i][2] + "_" + test_new_variables[i][0], 
                        str(prediction_no_zeros[i])]]

# output file to csv
with open("submission.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(submission)